In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input, BatchNormalization, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import Normalizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

data_path = '/content/drive/MyDrive/DEP2'

Mounted at /content/drive


In [ ]:
data = pd.read_csv(f'{data_path}/cleaned.csv')

In [ ]:
data_train, data_val = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
x_train, y_train = data_train.drop('score', axis=1), data_train['score']
x_val, y_val = data_val.drop('score', axis=1), data_val['score']

In [ ]:
feature_count = x_train.shape[1]
feature_count

27

In [ ]:
normalizer = Normalizer()
x_train = normalizer.fit_transform(x_train)
x_val = normalizer.transform(x_val)

In [ ]:
def create_model(hidden_layers=1, neurons=1, activation='relu', optimizer='adam'):

  model = Sequential()
  model.add(Dense(neurons, input_dim=x_train.shape[1], activation=activation))
  for i in range(hidden_layers):
    model.add(Dense(neurons, activation=activation))
  model.add(Dense(1, activation='relu'))
  model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])
  return model

In [ ]:
def tune_model(x_train, y_train, x_val, y_val):
  early_stop = EarlyStopping(monitor='val_loss', patience=5)
  model = KerasRegressor(build_fn=create_model, epochs=100, batch_size=32, verbose=0)
  param_grid = {
      'hidden_layers': [1, 2, 3],
      'neurons': [i for i in range(128, 320, 32)],
      'activation': ['tanh', 'relu'],
      'optimizer': ['Adam', 'SGD', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl']
  }
  grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose=2)
  grid_result = grid.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[early_stop])
  print(f'Best score: {grid_result.best_score_}')
  print(f'Best parameters: {grid_result.best_params_}')
  return grid_result.best_params_

params = tune_model(x_train, y_train, x_val, y_val)

<ipython-input-9-58a1f585d916>:3: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasRegressor(build_fn=create_model, epochs=100, batch_size=32, verbose=0)


Fitting 3 folds for each of 288 candidates, totalling 864 fits
[CV] END activation=tanh, hidden_layers=1, neurons=128, optimizer=Adam; total time=   3.6s
[CV] END activation=tanh, hidden_layers=1, neurons=128, optimizer=Adam; total time=   1.7s
[CV] END activation=tanh, hidden_layers=1, neurons=128, optimizer=Adam; total time=   1.3s
[CV] END activation=tanh, hidden_layers=1, neurons=128, optimizer=SGD; total time=   1.1s
[CV] END activation=tanh, hidden_layers=1, neurons=128, optimizer=SGD; total time=   1.7s
[CV] END activation=tanh, hidden_layers=1, neurons=128, optimizer=SGD; total time=   1.1s
[CV] END activation=tanh, hidden_layers=1, neurons=128, optimizer=RMSprop; total time=   1.8s
[CV] END activation=tanh, hidden_layers=1, neurons=128, optimizer=RMSprop; total time=   1.7s
[CV] END activation=tanh, hidden_layers=1, neurons=128, optimizer=RMSprop; total time=   1.8s
[CV] END activation=tanh, hidden_layers=1, neurons=128, optimizer=Adadelta; total time=   1.6s
[CV] END activati

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=5)
model = create_model(**params)
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs = 25, callbacks=[early_stop])

Epoch 1/25
100/100 [==============================] - 1s 3ms/step - loss: 53.8647 - mae: 5.9059 - val_loss: 27.9242 - val_mae: 4.5788
Epoch 2/25
100/100 [==============================] - 0s 2ms/step - loss: 26.7090 - mae: 4.4734 - val_loss: 29.0737 - val_mae: 4.6576
Epoch 3/25
100/100 [==============================] - 0s 2ms/step - loss: 26.7812 - mae: 4.4829 - val_loss: 27.9343 - val_mae: 4.5796
Epoch 4/25
100/100 [==============================] - 0s 2ms/step - loss: 26.7955 - mae: 4.4816 - val_loss: 27.7673 - val_mae: 4.5690
Epoch 5/25
100/100 [==============================] - 0s 2ms/step - loss: 26.7574 - mae: 4.4828 - val_loss: 28.0751 - val_mae: 4.5866
Epoch 6/25
100/100 [==============================] - 0s 2ms/step - loss: 26.8015 - mae: 4.4795 - val_loss: 28.4059 - val_mae: 4.6055
Epoch 7/25
100/100 [==============================] - 0s 2ms/step - loss: 26.7313 - mae: 4.4793 - val_loss: 27.6283 - val_mae: 4.5546
Epoch 8/25
100/100 [==============================] - 0s 2ms/s

In [ ]:
data_test = pd.read_csv(f'{data_path}/test.csv')
x_test, y_test = data_test.drop('score', axis=1), data_test['score']
x_test = normalizer.transform(x_test)





In [ ]:
for input, output in zip(x_test[0:10], y_test[0:10]):
  input = input.reshape((1,feature_count))

  print(model.predict(input)[0][0], output)

1/1 [==============================] - 0s 74ms/step
13.669489 7.0
1/1 [==============================] - 0s 20ms/step
13.713525 29.0
1/1 [==============================] - 0s 15ms/step
13.693249 18.0
1/1 [==============================] - 0s 16ms/step
13.669498 6.0
1/1 [==============================] - 0s 15ms/step
13.716128 10.0
1/1 [==============================] - 0s 14ms/step
13.66951 8.0
1/1 [==============================] - 0s 16ms/step
13.728631 18.0
1/1 [==============================] - 0s 14ms/step
13.699937 14.0
1/1 [==============================] - 0s 14ms/step
13.669478 23.0
1/1 [==============================] - 0s 15ms/step
13.757205 24.0


In [ ]:
model.save(f'{data_path}/model.h5')